In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

src = "/content/drive/MyDrive/data/fake_real_news/"

In [16]:
# Load the data

fake_df = pd.read_csv(src + "Fake.csv")
true_df = pd.read_csv(src + "True.csv")

print("Fake news articles : " + str(len(fake_df)))
print("True news articles : " + str(len(true_df)))

fake_df["category"] = 0
true_df["category"] = 1

news_df = pd.concat([fake_df, true_df])

Fake news articles : 23481
True news articles : 21417


In [19]:
#Any blanks ?

news_df.isna().sum()

title       0
text        0
subject     0
date        0
category    0
dtype: int64

In [20]:
#Create the train and test splits
x_train, x_test, y_train, y_test = train_test_split(news_df.text, news_df.category)